<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/Reducing%20Commercial%20Aviation%20Fatalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os 
import numpy as np
import zipfile
from tqdm import tqdm
from matplotlib import pyplot as plt
from pandas import DataFrame
import pandas as pd
RANDOM_SEED = 42

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls -l "drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities"

total 439092
-rw------- 1 root root 449629430 Mar 17 03:41 train.csv.zip


In [7]:
!unzip "drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities/train.csv.zip"

Archive:  drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities/train.csv.zip
  inflating: train.csv               


In [12]:
!ls -lrt /content/

total 1205840
---------- 1 root root 1234764001 Dec 14 23:11 train.csv
drwxr-xr-x 1 root root       4096 Apr  4 20:20 sample_data
drwx------ 3 root root       4096 Apr 10 03:55 drive


In [0]:
df = pandas.read_table('train.csv', delim_whitespace=True, names=('id', 'crew', 'experiment', 'time', 'seat', 'eeg_fp1', 'eeg_f7', 'eeg_f8', 'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3', 'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz', 'eeg_c3', 'eeg_cz', 'eeg_o2'))